In [1]:
import onnxruntime as ort
import numpy as np
import pandas as pd

def load_onnx_model(onnx_file_path):
    """
    Loads a saved ONNX model from the given file path.

    Parameters:
    - onnx_file_path: str - Path to the ONNX model file.

    Returns:
    - onnxruntime.InferenceSession - Loaded ONNX model.
    """
    try:
        session = ort.InferenceSession(onnx_file_path)
        print(f"ONNX model loaded successfully from {onnx_file_path}")
        return session
    except Exception as e:
        print(f"Failed to load ONNX model from {onnx_file_path}: {e}")
        return None

def predict_with_onnx(session, X):
    """
    Runs predictions using the loaded ONNX model.

    Parameters:
    - session: onnxruntime.InferenceSession - Loaded ONNX model session.
    - X: pd.DataFrame or np.array - Input features for prediction.

    Returns:
    - np.array - Model predictions.
    """
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()  # Convert DataFrame to numpy array
    
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    
    preds = session.run([output_name], {input_name: X.astype(np.float32)})[0]
    return preds

In [2]:
from single_cat_model import SingleCategoryModel

model_placeholder = SingleCategoryModel(category_number=1)

# Load the ONNX model
onnx_model_path = "./models/onnx/category_1_model.onnx"
onnx_session = load_onnx_model(onnx_model_path)

# prepare the data
df = pd.read_json('../data/dataset.json').drop(columns=['sold_price'])
df = model_placeholder.preprocess_data(df=df)

df.head()

Initialized SingleCategoryModel for category 1.
ONNX model loaded successfully from ./models/onnx/category_1_model.onnx


,price,view_count,steam_item_id,steam_cs2_profile_rank,steam_level,steam_friend_count,steam_dota2_solo_mmr,steam_converted_balance,steam_pubg_inv_value,steam_cs2_inv_value,...,steam_cs2_last_launched_minute,steam_cs2_last_launched_second,steam_cs2_last_launched_weekday,inv_value_sum,game_count_sum,level_sum,price_per_view,total_steam_games,total_playtime,std_playtime
0,597,124,70108402,4,10,23,0,0,-1,130,...,12.0,27.0,3.0,1481,203,10,4.814516,125,1101.26,36.607566
1,75,17,71126523,2,0,0,0,0,-1,243,...,34.0,43.0,3.0,480,2,0,4.411765,1,0.00,0.000000
2,1000,92,69083816,17,20,27,0,8,-1,18,...,11.0,55.0,4.0,199,17,23,10.869565,9,256.22,67.700112
3,200,17,70902299,0,4,1,0,0,-1,0,...,0.0,0.0,0.0,112,20,3,11.764706,12,14.21,3.395692
4,120,25,71066650,3,2,3,0,0,-1,90,...,44.0,52.0,4.0,174,11,2,4.800000,8,83.58,30.862671


In [4]:
# Predict using the ONNX model
if onnx_session is not None:
    predictions = predict_with_onnx(onnx_session, df)
    print("Predictions:", predictions)

2024-11-21 19:29:27.023491 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {129624,1} for output predictions


Predictions: [[585.62506  ]
 [ 67.792694 ]
 [963.65906  ]
 ...
 [435.72034  ]
 [ -1.0241394]
 [373.00357  ]]
